In [ ]:
import csv
import io
import os

import boto3
import oracledb

# データベース接続情報
DB_USER = os.environ.get("DB_USER")
DB_PASSWORD = os.environ.get("DB_PASSWORD")
DB_HOST = os.environ.get("DB_HOST")
DB_PORT = os.environ.get("DB_PORT")
DB_SERVICE_NAME = os.environ.get("DB_SERVICE_NAME")
print(DB_USER, DB_PASSWORD, DB_HOST, DB_PORT, DB_SERVICE_NAME)

# S3接続情報
S3_BUCKET = os.environ.get("S3_BUCKET")
S3_KEY = os.environ.get("S3_KEY")

# データベースに接続
dsn = oracledb.makedsn(DB_HOST, DB_PORT, service_name=DB_SERVICE_NAME)
connection = oracledb.connect(user=DB_USER, password=DB_PASSWORD, dsn=dsn)

In [ ]:
# カーソルを作成
cursor = connection.cursor()

# SQLクエリを実行
cursor.execute("SELECT * FROM m_user")

# メモリ上でCSVデータを生成
csv_buffer = io.StringIO()
csvwriter = csv.writer(csv_buffer)
csvwriter.writerow([desc[0] for desc in cursor.description])  # ヘッダーを書き込む
for row in cursor:
    csvwriter.writerow(row)

In [ ]:
# S3にアップロード
s3_client = boto3.client("s3")
s3_client.put_object(Bucket=S3_BUCKET, Key=S3_KEY, Body=csv_buffer.getvalue())

print(f"File uploaded to S3: s3://{S3_BUCKET}/{S3_KEY}")